In [1]:
from utils import augment
from utils.augment import *

from trainer import *

import jax.numpy as jnp
import jax
import flax.linen as nn
import numpy as np
import random
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
from utils.dataloader import mel_dataset

wandb.init(
project='CLR',
entity='aiffelthon'
)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-09-20 03:50:42.218564: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-09-20 03:50:42.245538: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-20 03:50:42.917053: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-09-20 03:50:42.917168: W 

In [2]:
data = mel_dataset('/mnt/disks/sdb/dev_dataset', 'total')

Load song_meta.json...


100%|███████████████████████████████| 707989/707989 [00:01<00:00, 656052.75it/s]


Load complete!

Load file list...


4it [00:00, 21.46it/s]


In [3]:
mix = MixupBYOLA()
crop = RandomResizeCrop()

In [4]:
def collate_batch(batch):
    x_train_1 = []
    x_train_2 = []
    
    for x, y in batch:
        x = (np.array(x)+127)/100
        x = np.expand_dims(x, axis=-1)
        x = crop(mix(x))        
        x_train_1.append(x)
        
    for x, y in batch:
        x = (np.array(x)+127)/100
        x = np.expand_dims(x, axis=-1)
        x = crop(mix(x))        
        x_train_2.append(x)
            
    y_train = [y for _, y in batch]           
    return augment.post_norm(np.stack(x_train_1 + x_train_2, axis=0)), np.array(y_train)

In [5]:
dataset_size = len(data)
train_size = int(dataset_size * 0.8)    
test_size = dataset_size - train_size

train_dataset, test_dataset, = random_split(data, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=0, collate_fn=collate_batch)

In [6]:
def train_simclr(num_epochs, **kwargs):
    # Create a trainer module with specified hyperparameters
    trainer = AudioEncoderTrainer(exmp=jnp.ones((128,48,1876,1)))
    trainer.train_model(train_dataloader, test_dataloader, num_epochs=num_epochs)

    return trainer

In [7]:
simclr_trainer = train_simclr(num_epochs=1)

tcmalloc: large alloc 1660436480 bytes == 0x8fa3a000 @  0x7f738513c680 0x7f738515d824 0x7f7231f8e045 0x7f7231fa4441 0x7f7231fa53b0 0x7f7231fa5dab 0x7f723202eadb 0x7f723202ff59 0x7f7232030718 0x7f73679ae096 0x7f7231819acd 0x7f72317532a9 0x7f723151c7fd 0x7f7231505781 0x5f6929 0x5f74f6 0x50c383 0x570b26 0x569dba 0x5f6eb3 0x5f8892 0x66931d 0x5f627e 0x56d2d5 0x569dba 0x5f6eb3 0x5f6082 0x56d2d5 0x569dba 0x5f6eb3 0x5f8892
tcmalloc: large alloc 1660436480 bytes == 0x84a58000 @  0x7f738513c680 0x7f738515d824 0x7f7231f8e045 0x7f7231fa4441 0x7f7231fa53b0 0x7f7231fa5dab 0x7f723202eadb 0x7f723202ff59 0x7f7232030718 0x7f73679ae096 0x7f7231819acd 0x7f72317532a9 0x7f723151c7fd 0x7f7231505781 0x5f6929 0x5f74f6 0x50c383 0x570b26 0x569dba 0x5f6eb3 0x5f8892 0x66931d 0x5f627e 0x56d2d5 0x569dba 0x5f6eb3 0x5f6082 0x56d2d5 0x569dba 0x5f6eb3 0x5f8892
tcmalloc: large alloc 1660436480 bytes == 0x84a58000 @  0x7f738513c680 0x7f738515d824 0x7f7231f8e045 0x7f7231fa4441 0x7f7231fa53b0 0x7f7231fa5dab 0x7f723202eadb 0

In [14]:
def numpy_collate(batch):
    if isinstance(batch[0], np.ndarray):
        return np.stack(batch)
    elif isinstance(batch[0], (tuple,list)):
        transposed = zip(*batch)
        return [numpy_collate(samples) for samples in transposed]
    else:
        return np.array(batch)

In [15]:
class NumpyDataset(mel_dataset):
    # data.TensorDataset for numpy arrays

    def __init__(self, *arrays):
        self.arrays = arrays

    def __len__(self):
        return self.arrays[0].shape[0]

    def __getitem__(self, idx):
        return [arr[idx] for arr in self.arrays]

In [16]:
def eval_collate_batch(batch):
    x_train = [(np.array(x)+127)/100 for x, _ in batch]
    x_train = [np.expand_dims(np.array(x), axis=-1) for x, _ in batch]
    y_train = [y for _, y in batch]                 
        
    return np.array(x_train), np.array(y_train)

In [17]:
def prepare_data_features(encode_fn, data):
    # Encode all images
    dataset = DataLoader(data, batch_size=64,
                                  shuffle=True,
                                  drop_last=True,
                                  num_workers=0,
                                  collate_fn=eval_collate_batch)
    
    feats, labels = [], []
    for batch_imgs, batch_labels in tqdm(dataset):
        batch_feats = encode_fn(batch_imgs)
        feats.append(jax.device_get(batch_feats))
        labels.append(batch_labels)
        print(batch_feats.shape)
        print(batch_labels.shape)
        print(batch_imgs.shape)


    feats = np.concatenate(feats, axis=0)
    labels = np.concatenate(labels, axis=0)

    # # Sort images by labels for easier postprocessing later
    # idxs = labels.argsort()
    # labels, feats = labels[idxs], feats[idxs]

    return NumpyDataset(feats, labels)

In [18]:
dataset = DataLoader(train_dataset, batch_size=64,
                              shuffle=True,
                              drop_last=True,
                              num_workers=0,
                              collate_fn=eval_collate_batch)

In [ ]:
np.array(test_feats_simclr[0][0]).shape

In [ ]:
feats, labels = [], []

In [19]:
simclr_model = simclr_trainer.model.bind({'params': simclr_trainer.state.params,
                                          'batch_stats': simclr_trainer.state.batch_stats},
                                        mutable=['batch_stats'])

In [20]:
encode_fn = jax.jit(lambda img: simclr_model.encode(img))

In [21]:
train_feats_simclr = prepare_data_features(encode_fn, train_dataset)
test_feats_simclr = prepare_data_features(encode_fn, test_dataset)

  5%|██                                          | 1/21 [00:09<03:06,  9.31s/it]

(64, 512)
(64, 30)
(64, 48, 1876, 1)


 10%|████▏                                       | 2/21 [00:12<01:46,  5.62s/it]

(64, 512)
(64, 30)
(64, 48, 1876, 1)


 14%|██████▎                                     | 3/21 [00:15<01:20,  4.48s/it]

...

(64, 512)
(64, 30)
(64, 48, 1876, 1)


 80%|████████████████████████████████████         | 4/5 [00:12<00:03,  3.07s/it]

(64, 512)
(64, 30)
(64, 48, 1876, 1)


100%|█████████████████████████████████████████████| 5/5 [00:15<00:00,  3.09s/it]

(64, 512)
(64, 30)
(64, 48, 1876, 1)


In [24]:
def train_logreg(batch_size, train_feats_data, test_feats_data, num_epochs=100, **kwargs):
    # Data loaders
    train_loader = DataLoader(train_feats_data,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   drop_last=True,
                                   generator=torch.Generator().manual_seed(42),
                                   collate_fn=numpy_collate)
    test_loader = DataLoader(test_feats_data,
                                  batch_size=batch_size,
                                  shuffle=False,
                                  drop_last=False,
                                  collate_fn=numpy_collate)

    # Create a trainer module with specified hyperparameters
    trainer = LGTrainer(exmp=next(iter(train_loader))[0],
                        # model_suffix=model_suffix,
                        **kwargs)
    trainer.train_model(train_loader, test_loader, num_epochs=num_epochs)

    # Test best model on train and validation set
    train_result = trainer.eval_model(train_loader, 100) 
    test_result = trainer.eval_model(test_loader,100)

    return trainer

In [25]:
trainer = train_logreg(batch_size=64,
                         train_feats_data=train_feats_simclr,
                         test_feats_data=test_feats_simclr,
                         num_classes=30,
                         lr=1e-3,
                         weight_decay=1e-3,
                         )